In [ ]:
from gym.envs.registration import register  # https://github.com/openai/gym/blob/master/gym/envs/registration.py
import gym
import numpy as np

MY_ENV_NAME='FrozenLakeNonskid4x4-v0'

# 0 = left; 1 = down; 2 = right;  3 = up
# reward of 1 if success, no reward otherwise
# done = True even if not success due to timestep limit

register(
    id=MY_ENV_NAME,
    entry_point='gym.envs.toy_text:FrozenLakeEnv',
    kwargs={'map_name': '8x8', 'is_slippery': False},
    reward_threshold=0.78)

env = gym.make(MY_ENV_NAME)

In [ ]:
class Node(object):
    """
    each state will be represented as a State object. we do this 
    because we want to track (a) total rewards and (b) play count.
    
    we need to track rewards and play count in order to calculate
    UCB score; we want to calculate UCB score in order to select 
    the most promising action.
    """
    def __init__(self, state):
        self.state = state
        self.q = 0  # win count
        self.n = 0  # play count


def _find_valid_action_state_pairs(s1, env):
    """return a list of valid action/s2 pairs for a given s1.
    """
    valid_actions = []
    
    # explore all four possible actions
    for a in range(3):  
        s2, _, _, _ = env.step(a)  # what would s2 look like if we took action a
        
        # scenario 1: action does not lead to a new state, so ignore
        if s2 == s1:
            pass
        
        # scenario 2: action leads to a new state so we will keep it
        else:
            valid_actions.append((a, s2))
        
        # return env to original state
        env.s = s1  
    return valid_actions
        

def _find_valid_actions(s, env):
    """return a list of valid actions for state s.
    """
    valid_actions = []
    
    # explore all four actions and see if they are valid
    for a in range(3):  
        s2, _, _, _ = env.step(a)  # what would s2 look like if we took action a
        
        # scenario 1: action does not lead to a new state, so ignore
        if s2 == s:
            pass
        
        # scenario 2: action leads to a new state so we will keep it
        else:
            valid_actions.append(a)
        
        # return env to original state
        env.s = s
    return valid_actions

    
def select_action_from_root(root, memory, env):
    """return an action from root. 
    
    if root is unexpanded then return actions that lead to unexplored 
    states. if root is expanded (all possible s2 from root s1 have 
    been explored) then return action that leads to s2 with highest 
    UCB score.
    """
    # step 1: find all actions that would lead to a valid state
    action_states = _find_valid_action_state_pairs(root, env)
    
    # step 2a: if we havent seen that s2 before, then go with it
    for ap in action_states:
        a = ap[0]
        s2 = ap[1]
        if s2 in memory.keys():
            pass
        else:
            return a  # this action leads to a state that we havent seen before
        
    # step 2b: if we've visited all possible s2 from root, return action that would lead to s2 with the highest UCB
    action = action_states[0][0]
    max_ucb = 0
    for ap in action_states:
        s1 = memory[root]
        s2 = memory[ap[1]]
        score = calculate_ucb(s1, s2)
        if score > max_ucb:
            action = ap[0]
            max_ucb = score  # new score to beat
    return action
    
def calculate_ucb(s1, s2):
    """compute ucb score between s1 and s2.
    """
    c_param = .14
    
    # hack in case s2 is a pit
    if s2.n == 0:
        s2.n = 1
        
    term_1 = (s2.q / s2.n)
    term_2 = c_param * np.sqrt((2 * np.log(s1.n) / s2.n))
    return  term_1 + term_2

In [ ]:
n_rollouts = 100
max_steps = 20

In [ ]:
root_state = 5  # which state do we start?

In [ ]:
# memory is a data structure (should it be restarted at each step?)
m = {root_state: Node(root_state)}  # track states that we've seen before since we cant "color" environment

In [ ]:
for rollout in range(n_rollouts):
    s = root_state
    trace = [s]
    env.s = s  # manually set env to the state we want
    done = False
    steps = 0
    
    # the first action from root is determined differently from other actions, since it relies on ucb score
    a = select_action_from_root(s, m, env)

    while not done and steps < max_steps:
        s, r, done, _ = env.step(a)
        steps += 1

        # if we've seen state s2 before, we dont need to do anything
        if s in m:
            pass
        # if we havent seen state s2 before, we update memory and continue
        else:
            m[s] = Node(s)

        # add action to trace so we can eventually backpropagate from terminal through root
        trace.append(s)
        
        # select a new action
        if not done:
            a = select_action_from_root(s, m, env)

    # now that we're done with the rollout... backpropagate the nodes in the trajectory
    if r> 0:
        print("rollout", rollout, "trace", trace, "with reward", r)
    
    for t in set(trace):
        node = m[t]  # retrieve node for corresponding state
        node.q += r  # update reward
        node.n += 1  # update games played
print("-")

In [ ]:
# 0 = left; 1 = down; 2 = right;  3 = up
best_a = select_action_from_root(root_state, m, env)
s2, _, _, _ = env.step(best_a)
print("from state", root_state, ", our best move", best_a, "takes us to state", s2)
env.render()